In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# Filter is the block of data (pixels) to scan an image and produce a value
filter_size = (3,3)

# A grey scale image would have one, RGB would have 3
input_channel = 1

# Number of channels to produce. With data aggregated, each channel 
output_channel = 2

In [3]:
image = [
    [0.11, 0.12, 0.13, 0.14],
    [0.21, 0.22, 0.23, 0.24],
    [0.31, 0.32, 0.33, 0.34],
    [0.41, 0.42, 0.43, 0.44]
]


In [4]:
filter1 = [
    [.4, .6, .4],
    [.6, .9, .6],
    [.1, .6, .4],
]

filter2 = [
    [.9, .5, .9],
    [.5, .3, .5],
    [.9, .5, .9],
]

channel1 = [filter1]
channel2 = [filter2]

convolution_layer1 = [channel1, channel2]

print(convolution_layer1[0][0][2][0])

0.1


In [ ]:
for channel in convolution_layer1:
    for filter_layer in channel:
        pass
        # Run filter over images